# Assignment1 Search Engine

In [1]:
#import necessary library
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import brown
import time
import pandas as pd
import math

## 1. Load data

In [ ]:
# nltk.download('brown')

I use brown corpus dataset (author: W. N. Francis and H. Kucera)from https://www.nltk.org/nltk_data/

In [2]:
#1. tokenization
corpus = brown.sents(categories=['news'])

In [3]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) 

In [4]:
#there are 14394 words in the vocabs
len(vocabs)

14394

In [5]:
#mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}
index2word = {v:k for k, v in word2index.items()}

In [6]:
#add <UNK> as last index in the vocabs
vocabs.append('<UNK>')
word2index['<UNK>'] = 14394

## 2. Co-occurence Matrix X

for glove model

In [7]:
from collections import Counter
X_i = Counter(flatten(corpus))

In [8]:
skip_grams = []

for doc in corpus:
    for i in range(2, len(doc)-2):
        center = doc[i]
        outside = [doc[i-2],doc[i-1], doc[i+1],doc[i-2]]
        for each_out in outside:
            skip_grams.append((center, each_out))

In [9]:
X_ik_skipgrams = Counter(skip_grams)

### Weighting function

GloVe includes a weighting function to scale down too frequent words.

In [10]:
def weighting(w_i, w_j, X_ik):
    
    #check whether the co-occurences between w_i and w_j is available
    try:
        x_ij = X_ik[(w_i, w_j)]
    except:
        #if not exist, then set to 1 "laplace smoothing"
        x_ij = 1
        
    #set xmax
    x_max = 100
    #set alpha
    alpha = 0.75
    
    #if co-ocurrence does not exceeed xmax, then just multiply with some alpha
    if x_ij < x_max:
        result = (x_ij / x_max)**alpha
    #otherwise, set to 1
    else:
        result = 1
    
    return result

In [11]:
from itertools import combinations_with_replacement

X_ik = {} #keeping the co-occurences
weighting_dic = {} #scale the co-occurences using the weighting function

for bigram in combinations_with_replacement(vocabs, 2):
    if X_ik_skipgrams.get(bigram):  #if the pair exists in our corpus
        co = X_ik_skipgrams[bigram]
        X_ik[bigram] = co + 1 #for stability
        X_ik[(bigram[1], bigram[0])] = co + 1 #basically apple, banana = banana, apple
    else:
        pass
    
    weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)

## 3. Prepare train data

In [12]:
#create pairs of center word, and outside word for skipgram model

def random_batch(batch_size, corpus):

    skipgrams = []

    #loop each corpus
    for doc in corpus:
        #use window size=2
        for i in range(2, len(doc)-2):
            #center word
            center = word2index[doc[i]]
            #outside words = 4 words
            outside = (word2index[doc[i-2]],word2index[doc[i-1]], word2index[doc[i+1]],word2index[doc[i+2]],)
            #append outside words to a list
            for each_out in outside:
                skipgrams.append([center, each_out])
                
    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)
    
    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])
        
    return np.array(inputs), np.array(labels)
            
# x, y = random_batch(2, corpus)

In [13]:
#random batch for glove model
def random_batch_glove(batch_size, word_sequence, skip_grams, X_ik, weighting_dic):
    
    random_inputs, random_labels, random_coocs, random_weightings = [], [], [], []
    
    #convert our skipgrams to id
    skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]
    
    #randomly choose indexes based on batch size
    random_index = np.random.choice(range(len(skip_grams_id)), batch_size, replace=False)
    
    #get the random input and labels
    for index in random_index:
        random_inputs.append([skip_grams_id[index][0]])
        random_labels.append([skip_grams_id[index][1]])
        #coocs
        pair = skip_grams[index] 
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1
        random_coocs.append([math.log(cooc)])
    
        #weightings
        weighting = weighting_dic[pair]
        random_weightings.append([weighting])
        
    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weightings)

## Negative Sampling

In [15]:
#count
from collections import Counter

word_count = Counter(flatten(corpus))
word_count

#get the total number of words
num_total_words = sum([c for w, c in word_count.items()])
# num_total_words

In [18]:
unigram_table = []
z=0.01
for v in vocabs:
    uw = word_count[v] / num_total_words
    uw_alpha = int((uw ** 0.75) / z)
    unigram_table.extend([v] * uw_alpha)
    
# Counter(unigram_table)

## 3. Model

### without negative sampling

In [22]:
#there are 14395 words in vocabs
len(vocabs)
embedding = nn.Embedding(14395, 2)

In [24]:
# x_tensor = torch.LongTensor(x)
# embedding(x_tensor).shape  #(batch_size, 1, emb_size)

In [27]:
class Skipgram(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(Skipgram, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
    
    def forward(self, center, outside, all_vocabs):
        center_embedding     = self.embedding_center(center)  #(batch_size, 1, emb_size)
        outside_embedding    = self.embedding_center(outside) #(batch_size, 1, emb_size)
        all_vocabs_embedding = self.embedding_center(all_vocabs) #(batch_size, voc_size, emb_size)
        
        top_term = torch.exp(outside_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2))
        #batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1) 

        lower_term = all_vocabs_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2)
        #batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) = (batch_size, voc_size) 
        
        lower_term_sum = torch.sum(torch.exp(lower_term), 1)  #(batch_size, 1)
        
        loss = -torch.mean(torch.log(top_term / lower_term_sum))  #scalar
        
        return loss

    #add predict function to predict the model
    def predict(self, A, B, C):
        # Given three words (A, B, C), return their embeddings
        A_idx = torch.tensor([word2index[A]])  # Convert words to indices
        B_idx = torch.tensor([word2index[B]])
        C_idx = torch.tensor([word2index[C]])

        A_embedding = self.embedding_center(A_idx)
        B_embedding = self.embedding_center(B_idx)
        C_embedding = self.embedding_center(C_idx)
        
        return A_embedding, B_embedding, C_embedding
    
        

In [28]:
#prepare all vocabs

batch_size = 2
voc_size   = len(vocabs)

def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

all_vocabs = prepare_sequence(list(vocabs), word2index).expand(batch_size, voc_size)

### with negative sampling

In [29]:
import random

def negative_sampling(targets, unigram_table, k):
    batch_size = targets.shape[0]
    neg_samples = []
    for i in range(batch_size):  #(1, k)
        target_index = targets[i].item()
        nsample      = []
        while (len(nsample) < k):
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))
        
    return torch.cat(neg_samples) #batch_size, k

In [30]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(SkipgramNeg, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.logsigmoid        = nn.LogSigmoid()
    
    def forward(self, center, outside, negative):
        #center, outside:  (bs, 1)
        #negative       :  (bs, k)
        
        center_embed   = self.embedding_center(center) #(bs, 1, emb_size)
        outside_embed  = self.embedding_outside(outside) #(bs, 1, emb_size)
        negative_embed = self.embedding_outside(negative) #(bs, k, emb_size)
        
        uovc           = outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, 1)
        ukvc           = -negative_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, k)
        ukvc_sum       = torch.sum(ukvc, 1).reshape(-1, 1) #(bs, 1)
        
        loss           = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)
        
        return -torch.mean(loss)

    #predict function
    def predict(self, A, B, C):
        # Given three words (A, B, C), return their embeddings

        A_idx = torch.tensor([word2index[A]])  # Convert words to indices
        B_idx = torch.tensor([word2index[B]])
        C_idx = torch.tensor([word2index[C]])

        A_embedding = self.embedding_center(A_idx)
        B_embedding = self.embedding_center(B_idx)
        C_embedding = self.embedding_center(C_idx)
        
        return A_embedding, B_embedding, C_embedding

### Glove

In [31]:
class Glove(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(Glove, self).__init__()
        self.center_embedding  = nn.Embedding(voc_size, emb_size)
        self.outside_embedding = nn.Embedding(voc_size, emb_size)
        
        self.center_bias       = nn.Embedding(voc_size, 1) 
        self.outside_bias      = nn.Embedding(voc_size, 1)
    
    def forward(self, center, outside, coocs, weighting):
        center_embeds  = self.center_embedding(center) #(batch_size, 1, emb_size)
        outside_embeds = self.outside_embedding(outside) #(batch_size, 1, emb_size)
        
        center_bias    = self.center_bias(center).squeeze(1)
        target_bias    = self.outside_bias(outside).squeeze(1)
        
        inner_product  = outside_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #(batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1)
        
        loss = weighting * torch.pow(inner_product + center_bias + target_bias - coocs, 2)
        
        return torch.sum(loss)

    def predict(self, A, B, C):
        # Given three words (A, B, C), return their embeddings

        A_idx = torch.tensor([word2index[A]])  # Convert words to indices
        B_idx = torch.tensor([word2index[B]])
        C_idx = torch.tensor([word2index[C]])

        A_embedding = self.center_embedding(A_idx)
        B_embedding = self.center_embedding(B_idx)
        C_embedding = self.center_embedding(C_idx)
        
        return A_embedding, B_embedding, C_embedding

## 4. Training

In [34]:
batch_size = 2
emb_size   = 2
#skipgram
model1      = Skipgram(voc_size, emb_size)
#skipgram negative
model2      = SkipgramNeg(voc_size, emb_size)
#glove
model3      = Glove(voc_size, emb_size)
#use Adam as optimizer and use crossentropyloss
optimizer   = optim.Adam(model1.parameters(), lr=0.001)
optimizer   = optim.Adam(model2.parameters(), lr=0.001)
optimizer   = optim.Adam(model3.parameters(), lr=0.001)
criterion   = nn.CrossEntropyLoss()

In [35]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
num_epochs = 1000

start1 = time.time()
print("Model: Skipgram")
for epoch in range(num_epochs):
    #get batch
    input_batch, label_batch = random_batch(batch_size, corpus)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    #predict
    loss1 = model1(input_tensor, label_tensor, all_vocabs)
    #backprogate
    optimizer.zero_grad()
    loss1.backward()
    #update alpha
    optimizer.step()
    #print the loss
    
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1:6.0f} | Loss: {loss1:2.6f} ")
end1 = time.time()
epoch_mins1, epoch_secs1 = epoch_time(start1, end1)
print(f"time: {epoch_mins1}m {epoch_secs1}s")
print("-"*20)

start2 = time.time()
print("Model: Skipgram Negative Sampling")
for epoch in range(num_epochs):
    #get batch
    input_batch, label_batch = random_batch(batch_size, corpus)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    #predict
    k=5
    neg_samples = negative_sampling(label_tensor, unigram_table, k)
    loss2 = model2(input_tensor, label_tensor, neg_samples)

    #backprogate
    optimizer.zero_grad()
    loss2.backward()
    #update alpha
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1:6.0f} | Loss: {loss2:2.6f} ")
end2 = time.time()
epoch_mins2, epoch_secs2 = epoch_time(start2, end2)
print(f"time: {epoch_mins2}m {epoch_secs2}s")
print("-"*20)

start3 = time.time()
print("Model: Glove")
for epoch in range(num_epochs):

    input_batch, target_batch, cooc_batch, weighting_batch = random_batch_glove(batch_size, corpus, skip_grams, X_ik, weighting_dic)
    input_batch  = torch.LongTensor(input_batch)         #[batch_size, 1]
    target_batch = torch.LongTensor(target_batch)        #[batch_size, 1]
    cooc_batch   = torch.FloatTensor(cooc_batch)         #[batch_size, 1]
    weighting_batch = torch.FloatTensor(weighting_batch) #[batch_size, 1]
    
    optimizer.zero_grad()
    loss3 = model3(input_batch, target_batch, cooc_batch, weighting_batch)
    
    loss3.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1:6.0f} | Loss: {loss3:2.6f} ")
end3 = time.time()
epoch_mins3, epoch_secs3 = epoch_time(start3, end3)
print(f"time: {epoch_mins3}m {epoch_secs3}s")
print("-"*20)


Model: Skipgram
Epoch    100 | Loss: 11.108609 
Epoch    200 | Loss: 9.449618 
Epoch    300 | Loss: 9.931557 
Epoch    400 | Loss: 9.949294 
Epoch    500 | Loss: 13.314457 
Epoch    600 | Loss: 10.792645 
Epoch    700 | Loss: 9.351409 
Epoch    800 | Loss: 9.354146 
Epoch    900 | Loss: 13.083500 
Epoch   1000 | Loss: 9.689467 
time: 10m 33s
--------------------
Model: Skipgram Negative Sampling
Epoch    100 | Loss: 2.268116 
Epoch    200 | Loss: 1.914912 
Epoch    300 | Loss: 3.088072 
Epoch    400 | Loss: 1.093648 
Epoch    500 | Loss: 1.268568 
Epoch    600 | Loss: 0.778361 
Epoch    700 | Loss: 3.887585 
Epoch    800 | Loss: 3.546030 
Epoch    900 | Loss: 1.530818 
Epoch   1000 | Loss: 1.979582 
time: 10m 28s
--------------------
Model: Glove
Epoch    100 | Loss: 1.102688 
Epoch    200 | Loss: 0.432289 
Epoch    300 | Loss: 0.366801 
Epoch    400 | Loss: 10.537242 
Epoch    500 | Loss: 0.557471 
Epoch    600 | Loss: 0.035286 
Epoch    700 | Loss: 1.085876 
Epoch    800 | Loss: 0.03

## Model comparison and analysis

data source: https://www.fit.vutbr.cz/~imikolov/rnnlm/word-test.v1.txt
for semantic.txt: capital-common-countries
for syntacticdataset.txt: past-tense

In [42]:
#import analogies dataset in order to calculate syntactic and semantic accuracy
analogies_df = pd.read_csv('syntacticdataset.txt', header=None)
analogies_df[['1','2','3','4']] = analogies_df[0].str.split(' ', expand=True)
analogies_df.drop([0], axis='columns', inplace=True)
analogies_df=analogies_df.values.tolist()

In [45]:
analogies_df2= pd.read_csv('semantic.txt', header=None)
analogies_df2[['1','2','3','4']] = analogies_df2[0].str.split(' ', expand=True)
analogies_df2.drop([0], axis='columns', inplace=True)
analogies_df2=analogies_df2.values.tolist()

In [43]:
#syntactic accuracy function
def calculate_syntactic_accuracy(model, analogy_dataset):
    correct_predictions = 0

    for analogy_question in analogy_dataset:
        try:

            A, B, C, correct_answer = analogy_question  # extract components into question and answer model should predict
            if model == 'model_gensim':
                predicted_answer = model.most_similar(positive=[A, B, C])
            else:
                predicted_answer = model.predict(A, B, C)  # predict the answer from previous models

                if predicted_answer == correct_answer:
                    correct_predictions += 1

        except:
            continue

    syntactic_accuracy = (correct_predictions / len(analogy_dataset)) * 100
    return syntactic_accuracy


In [44]:
#semantic accuracy function
def calculate_semantic_accuracy(model, analogy_dataset, vocab):
    correct_predictions = 0

    for analogy_question in analogy_dataset:
        try:
            A, B, C, correct_answer = analogy_question 

            # Check if words are in the vocabulary before using them
            if (A in word2index) and (B in word2index) and( C in word2index):
                A_idx, B_idx, C_idx = word2index[A], word2index[B], word2index[C]

                # Predict the word that completes the analogy
                predicted_answer = model.predict(A_idx, B_idx, C_idx)

                # Convert the predicted answer index to the corresponding word
                predicted_word = next(key for key, value in word2index.items() if value == predicted_answer.item())

                if predicted_word == correct_answer:
                    correct_predictions += 1
        except:
            continue

    semantic_accuracy = (correct_predictions / len(analogy_dataset)) * 100
    return semantic_accuracy


In [46]:
syntactic_accuracy1 =calculate_syntactic_accuracy(model1, analogies_df)
syntactic_accuracy2 =calculate_syntactic_accuracy(model2, analogies_df)
syntactic_accuracy3 =calculate_syntactic_accuracy(model3, analogies_df)
print(f"Syntactic Accuracy Skipgram: {syntactic_accuracy1}%")
print(f"Syntactic Accuracy SkipgramNeg: {syntactic_accuracy2}%")
print(f"Syntactic Accuracy Glove: {syntactic_accuracy3}%")

Syntactic Accuracy Skipgram: 0.0%
Syntactic Accuracy SkipgramNeg: 0.0%
Syntactic Accuracy Glove: 0.0%


In [47]:
semantic_accuracy = calculate_semantic_accuracy(model1, analogies_df2, word2index)
semantic_accuracy2 = calculate_semantic_accuracy(model2, analogies_df2, word2index)
semantic_accuracy3 = calculate_semantic_accuracy(model3, analogies_df2, word2index)
print(f"Semantic Accuracy: {semantic_accuracy}%")
print(f"Semantic Accuracy2: {semantic_accuracy2}%")
print(f"Semantic Accuracy3: {semantic_accuracy3}%")

Semantic Accuracy: 0.0%
Semantic Accuracy2: 0.0%
Semantic Accuracy3: 0.0%


## Test the model

I selected Glove model because the model have the lowest lost and spend the fewest time

In [48]:
def get_embed(model, word):
    id_tensor = torch.LongTensor([word2index[word]])
    v_embed = model.center_embedding(id_tensor)
    u_embed = model.outside_embedding(id_tensor)
    word_embed = (v_embed + u_embed) / 2
    x, y = word_embed[0][0].item(), word_embed[0][1].item()
    return x, y

In [49]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

source of ttb_wiki data: https://en.wikipedia.org/wiki/The_Big_Bang_Theory

In [51]:
my_file = open("tbbt_wiki.txt", "r") 
  
# reading the file containg information
data = my_file.read()
#convert data into list
data_into_list = data.replace('\n', ' ').split(".") 

# printing the data 
print(data_into_list) 
my_file.close() 

['The Big Bang Theory is an American television sitcom created by Chuck Lorre and Bill Prady, both of whom served as executive producers and head writers on the series, along with Steven Molaro', ' It aired on CBS from September 24, 2007, to May 16, 2019, running for 12 seasons and 279 episodes', "[3]  The show originally centered on five characters living in Pasadena, California: Leonard Hofstadter (Johnny Galecki) and Sheldon Cooper (Jim Parsons), both physicists at Caltech, who share an apartment; Penny (Kaley Cuoco), a waitress and aspiring actress who lives across the hall; and Leonard and Sheldon's similarly geeky and socially awkward friends and coworkers, aerospace engineer Howard Wolowitz (Simon Helberg) and astrophysicist Raj Koothrappali (Kunal Nayyar)", '[4][5] Over time, supporting characters were promoted to starring roles, including neuroscientist Amy Farrah Fowler (Mayim Bialik), microbiologist Bernadette Rostenkowski (Melissa Rauch), and comic book store owner Stuart B

In [56]:
from nltk.tokenize import word_tokenize

#tokennize function
def tokenize_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Tokenize the words using nltk's word_tokenize function
    words = word_tokenize(text)

    return words


file_path = 'tbbt_wiki.txt'
tokenized_words = tokenize_text(file_path)
tbbt_vocab = set(tokenized_words)

# print("vocabs:", tbbt_vocab)


vocabs: {'fails', '26', 'eleventh', 'returns', 'The', 'show', 'geeky', 'which', 'whether', 'doctorate', 'evolution', '19', 'five-week', 'Batgirl', 'hold', 'slot', 'Greene', 'pages', 'because', 'Stuart', 'wedding', 'Her', 'night', 'expectation', 'calls', 'income', 'microbiologist', 'record', 'version', 'Howard', 'Laurie', 'romantic', '4.4', 'supporting', '62', '15', '128', 'progresses', '74', 'provide', 'Mr.', 'intimate', 'An', 'marked', 'America', '24', 'sung', 'Nye', 'company', 'grasp', 'fifth', 'if', 'ground', 'high-profile', 'ceremony', 'ranked', 'fascinating', 'fuel', 'CraveTV', 'their', 'artists', '42', 'order', 'School', 'Vegas', 'herself', 'ever', 'Pilot', 'Among', 'led', 'William', 'New', 'universe', 'named', 'Oedipal', 'unofficial', '40', 'laureate', 'movie', 'assistance', 'memory', 'oddly', 'rest', 'young', 'does', 'book', 'Doctor', 'home', 'cut', 'eleven', 'Hamill', 'premiering', 'something', 'possesses', 'dialogue', 'controversial', 'between', 'neuroscience', 'via', '14.22'

In [57]:
#function that search for the 10th most similar words of input word
def search_similar(input):
    word_dict={}
    try:
        #use glove model
        input_embed = get_embed(model3, input)
        for word in tbbt_vocab:
            if word in vocabs:
                word_dict[word] =  cos_sim(input_embed, get_embed(model3, word))
            else:
                continue

        sorted_dict = dict(sorted(word_dict.items(), key=lambda item: item[1], reverse=True))
    except:
        print('There is no word in the dictionary, please type new word')

    return list(sorted_dict)[:10]
        

In [59]:
#for example, I search the word 'city'
x = search_similar('city')
print(x)

['alternative', 'works', 'share', 'operated', 'won', 'renewed', 'could', 'many', 'else', '85']


In [60]:

# Specify the file path where you want to save the model parameters
param_file_path = 'model_gl.bin'

# Save only the parameters of the model in binary format
torch.save(model3.state_dict(), param_file_path)